## Date Grouping

Notebook for developing methods to infer groups of measurements based on patterns in time differences between them

In [1]:
import pandas as pd
import numpy as np

In [37]:
dates = pd.Series([
    '2018-06-27 14:11:00', 
    '2018-06-27 14:12:01', 
    '2018-06-27 14:12:01', 
    '2018-06-27 14:13:00', 
    '2018-06-28 11:50:00',
    '2018-06-28 11:50:30',
    '2018-06-28 11:51:00',
    '2018-06-28 11:51:59'
])
dates = pd.to_datetime(dates)
dates

0   2018-06-27 14:11:00
1   2018-06-27 14:12:01
2   2018-06-27 14:12:01
3   2018-06-27 14:13:00
4   2018-06-28 11:50:00
5   2018-06-28 11:50:30
6   2018-06-28 11:51:00
7   2018-06-28 11:51:59
dtype: datetime64[ns]

In [38]:
diffs = dates.diff()
diffs

0        NaT
1   00:01:01
2   00:00:00
3   00:00:59
4   21:37:00
5   00:00:30
6   00:00:30
7   00:00:59
dtype: timedelta64[ns]

In [39]:
groups = (dates.diff().dt.seconds >= 3600).cumsum()
groups.map(dates.groupby(groups).min())

0   2018-06-27 14:11:00
1   2018-06-27 14:11:00
2   2018-06-27 14:11:00
3   2018-06-27 14:11:00
4   2018-06-28 11:50:00
5   2018-06-28 11:50:00
6   2018-06-28 11:50:00
7   2018-06-28 11:50:00
dtype: datetime64[ns]

In [40]:
diffs.dt.seconds

0        NaN
1       61.0
2        0.0
3       59.0
4    77820.0
5       30.0
6       30.0
7       59.0
dtype: float64

In [51]:
x = pd.Series(['a', 'b', 'b'])

In [57]:
y = []
y.extend([])

In [41]:
v = diffs.iloc[0]

In [42]:
v

NaT

In [43]:
v.seconds

nan

In [44]:
dates.is_monotonic_increasing

True

In [45]:
dates.is_unique

False

In [46]:
dates.drop_duplicates().is_unique

True

In [34]:
def group_dates(dates, min_gap_seconds):
    # Make sure dates are sorted
    if not dates.is_monotonic_increasing:
        dates = dates.sort_values()
        
    # Create a new group index each time the difference between steps exceeds the given threshold (in seconds)
    groups = (dates.diff().dt.seconds >= min_gap_seconds).cumsum()
    
    # Get the minimum date for each group and then get a vector of len(dates) continaing the group date
    # for each original date
    groups = groups.map(dates.groupby(groups).min())
    
    # Return a series mapping the original dates to the grouped date
    return pd.Series(groups.values, index=dates.values)

In [35]:
group_dates(dates, 3600)

0   2018-06-27 14:11:00
1   2018-06-27 14:11:00
2   2018-06-27 14:11:00
3   2018-06-28 11:50:00
4   2018-06-28 11:50:00
5   2018-06-28 11:50:00
6   2018-06-28 11:50:00
dtype: datetime64[ns]


2018-06-27 14:11:00   2018-06-27 14:11:00
2018-06-27 14:12:01   2018-06-27 14:11:00
2018-06-27 14:13:00   2018-06-27 14:11:00
2018-06-28 11:50:00   2018-06-28 11:50:00
2018-06-28 11:50:30   2018-06-28 11:50:00
2018-06-28 11:51:00   2018-06-28 11:50:00
2018-06-28 11:51:59   2018-06-28 11:50:00
dtype: datetime64[ns]